https://docs.microsoft.com/ko-kr/azure/cognitive-services/custom-vision-service/quickstarts/image-classification?tabs=visual-studio&pivots=programming-language-python#upload-and-tag-images

In [1]:
pip install azure-cognitiveservices-vision-customvision

Note: you may need to restart the kernel to use updated packages.


In [11]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

아래 내용들 찾는법
 
ENDPOINT : 포탈의 키 및 엔드포인트
training_key : 포탈의 키 및 엔드포인트
prediction_resource_id : 포털에서 커스텀비전 들어가서 속성

In [12]:
ENDPOINT = "https://mledu-cuv21.cognitiveservices.azure.com/"
PRE_ENDPOINT = "https://mleducuv21-prediction.cognitiveservices.azure.com/"
training_key = "be34b8372fea4de48f4d1bea2f3e8f03"
prediction_key = "f27f751a52c14cad9fa98021d0b7aa32"
prediction_resource_id = "/subscriptions/fe07f267-115d-404d-8c01-dc1d36f23a4c/resourceGroups/MLEDU21/providers/Microsoft.CognitiveServices/accounts/MLEDUCuV21-Prediction"

In [13]:
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(PRE_ENDPOINT, prediction_credentials)

In [14]:
publish_iteration_name = "classifyModel"

# Create a new project
print ("Creating project...")
project_name = uuid.uuid4()
project = trainer.create_project("minipjt_test")

Creating project...


In [15]:
# Make two tags in the new project
neoguri_tag = trainer.create_tag(project.id, "너구리")
charmcracker_tag = trainer.create_tag(project.id, "참크래커")

In [16]:
base_image_location = os.path.join (os.path.dirname("./"), "Images")

print("Adding images...")

image_list = []

for image_num in range(1, 11):
    file_name = "neoguri_{}.jpg".format(image_num)
    with open(os.path.join (base_image_location, "neoguri", "neoguri"+str(image_num)+".jpg"), "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[neoguri_tag.id]))

for image_num in range(1, 11):
    file_name = "charmcracker_{}.jpg".format(image_num)
    with open(os.path.join (base_image_location, "charmcracker", "charmcracker"+str(image_num)+".jpg"), "rb") as image_contents:
        image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[charmcracker_tag.id]))

upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=image_list))
if not upload_result.is_batch_successful:
    print("Image batch upload failed.")
    for image in upload_result.images:
        print("Image status: ", image.status)
    exit(-1)

Adding images...


In [17]:
print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    print ("Waiting 10 seconds...")
    time.sleep(10)

Training...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Waiting 10 seconds...
Training status: Training
Wa

In [9]:
import http.client, urllib.request, urllib.parse, urllib.error, base64

headers = {
    # Request headers
    'Training-key': prediction_key,
}

params = urllib.parse.urlencode({
})

try:
    conn = http.client.HTTPSConnection('japaneast.api.cognitive.microsoft.com')
    conn.request("GET", "/customvision/v3.3/Training/projects/5ebe1537-dad2-4ef4-b2a2-73fc3cf1af0c/iterations?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    print(data)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

b'{"error":{"code":"401","message": "The GetIterations Operation under Custom_Vision_Training_3.3 is not supported with the current subscription key and pricing tier Custom_Vision.Prediction.S0."}}'


In [18]:
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(PRE_ENDPOINT, prediction_credentials)

with open(os.path.join (base_image_location, "Test/test_image.jpg"), "rb") as image_contents:
    results = predictor.classify_image(project.id, publish_iteration_name, image_contents.read())

    # Display the results.
    for prediction in results.predictions:
        print("\t" + prediction.tag_name +
              ": {0:.2f}%".format(prediction.probability * 100))

CustomVisionErrorException: Invalid iteration

In [23]:
import requests
import json
import html
url="https://mledu-cuv21.cognitiveservices.azure.com/customvision/v3.0/Prediction/77d3d89e-3bf9-4367-a0ae-e6b2db459289/classify/iterations/Iteration1/image"
headers={'content-type':'application/octet-stream','Prediction-Key':'f27f751a52c14cad9fa98021d0b7aa32'}
response =requests.post(url,data=open("./Images/Test/test_image.jpg","rb"),headers=headers)
analysis = response.json()

In [24]:
print(analysis)

{'id': '5bd0c8e4-6f3b-4501-be66-683a53330f04', 'project': '77d3d89e-3bf9-4367-a0ae-e6b2db459289', 'iteration': '8e8d79ff-3789-4bb8-87e8-226ec4dfb07f', 'created': '2021-05-30T15:11:10.793Z', 'predictions': [{'probability': 0.99999905, 'tagId': '8669d6e7-e558-4d30-a266-bc02465baed6', 'tagName': '너구리'}, {'probability': 8.6426735e-07, 'tagId': 'cc4143d1-2aed-4e57-8f9b-83a5c06ce47a', 'tagName': '참크래커'}]}


In [26]:
print(analysis['predictions'][0]['tagName'])

너구리
